<img src="figures/rki-logo.jpg"
     alt="Robert Koch Institut"
     style="float: right; margin-right: 100px; margin-top:20px;" />

 <img src="figures/daki-fws.png"
     alt="DAKI-FWS"
     style="float: right; margin-right: 50px; margin-bottom:20px; height:80px; width:auto;"/>
<h1>Forecasting competition evaluation report</h1>
<p style="margin-top:70px">The evaluation of forecasting models is an essential step in the development of our early warning system. Here, we will assess the overall performance as well as performance over time and regions, which can help identify situations in which models may perform in an undesired fashion. Further, we will look whether under-, over-prediction, or the calibration with respect to uncertainty of the prediction are the main drivers of performance loss. This again is meant to help improve the model by identifying potential problems. Furthermore, the models will be assessed according to how well they specify the uncertainty of their predictions. Being able to correctly specify the uncertainty of the forecast is an essential requirement for infectious disease forecasts for the use in an early warning system.</p>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display
from math import ceil
import glob
from scripts import visualizations
from scripts import utils
from scripts import scoring
from scripts.generate_report import *

# load data
results = utils.load_results("./results/res.pickle")
results["model"] = results["team"] + "-" + results["model"]
results.drop(["team"], axis=1, inplace=True)

# execute code for tab navigation
display(HTML(js_tab_navigation))
display(HTML(css_tab_navigation))

In [ ]:
#utils.reset_results_file()

In [3]:
plt.ioff(); # turn matplotlib interactive mode off

In [2]:
def save_current(figure_name):
    """
    Saves current matplotlib figure to file, handles git commit
    """
    # save current pyplot figure
    plt.savefig(figure_name)
    # do git stuff

In [3]:
%matplotlib agg
    
wis_boxplot(aggregate_forecast_week(results, "wis"), "model", "forecast_period")
save_current("docs/figures/boxplot.png")

 # Performance Evaluation
 The performance of all forecasts will be evaluated by the (relative) weighted interval score [WIS](https://cran.r-project.org/web/packages/scoringutils/vignettes/metric-details.html) which is a proper scoring rule that considers not only the point estimate but the entire predictive distribution. A smaller WIS indicates a better performance. The WIS allows to distinguish between performance loss due to over-prediction, under-prediction, and sharpness.
 ![Boxplots of weighted interval score by model and forecasting period for the years 2021 - 23](./figures/boxplot.png)
 
***Fig 1: Comparison of forecasting model (submitted and baseline) performances in terms of (relative) WIS for the 4 different forecast horizons and the overall performance. Lower WIS indicates better performance. Boxplots are summaries of WIS scores for all counties and weeks for which a forecast was submitted. Models may differ in weeks and counties for which they have provided forecasts. Forecasts from baseline models exist for all counties and dates.***


**Table 1: (2021) Comparison of forecasting model (submitted and baseline) performances in terms of WIS for the 4 different forecast horizons and the overall performance. Here, the mean and 95% CI of the computed WIS scores is shown.**

In [4]:
tab1 = corresponding_boxplot_table(results[results.refdate < np.datetime64("2022-01-01")])
tab2 = corresponding_boxplot_table(results[(results.refdate >= np.datetime64("2022-01-01")) & (results.refdate <= np.datetime64("2022-12-31"))])
tab3 = corresponding_boxplot_table(results[results.refdate >= np.datetime64("2023-01-01")])
tab1

,model,complete score (CI),week 1 score (CI),week 2 score (CI),week 3 score (CI),week 4 score (CI)
0,HHI-ConnstantSpreadTest,4.38 (/),4.6 (/),5.08 (/),4.47 (/),3.36 (/)
1,HPI-baseMpnnLstm,9.73 (5.74 - 13.73),9.05 (5.71 - 12.39),10.42 (5.73 - 15.11),nan (/),nan (/)
2,HPI-decomp1,30.29 (16.83 - 43.76),27.38 (16.48 - 38.28),28.62 (16.54 - 40.71),31.41 (16.8 - 46.03),33.76 (17.01 - 50.51)
3,RKIsurv2-arima,17.58 (10.84 - 24.32),14.83 (9.5 - 20.16),17.08 (10.43 - 23.72),19.17 (11.48 - 26.85),19.24 (11.75 - 26.72)
4,RKIsurv2-etsAAA,16.93 (9.14 - 24.73),14.03 (8.06 - 19.99),17.05 (8.9 - 25.21),18.77 (9.54 - 28.01),17.88 (9.76 - 26.0)
5,RKIsurv2-etsAAN,17.41 (9.53 - 25.3),14.35 (8.25 - 20.44),17.44 (9.09 - 25.79),19.21 (9.81 - 28.6),18.65 (10.58 - 26.71)
6,RKIsurv2-etsANN,17.12 (9.62 - 24.61),14.37 (8.58 - 20.15),17.08 (9.63 - 24.53),18.73 (10.06 - 27.39),18.29 (10.0 - 26.58)


**Table 2: (2022) Comparison of forecasting model (submitted and baseline) performances in terms of WIS for the 4 different forecast horizons and the overall performance. Here, the mean and 95% CI of the computed WIS scores is shown.**

In [5]:
tab2

,model,complete score (CI),week 1 score (CI),week 2 score (CI),week 3 score (CI),week 4 score (CI)
0,HHI-seq2seq4096d14,3300.33 (/),3520.98 (/),3370.79 (/),2965.76 (/),3343.78 (/)
1,HHI-seq2seq4096d21,3320.37 (/),3540.25 (/),3392.25 (/),2982.02 (/),3366.98 (/)
2,HHI-seq2seq4096d28,3319.39 (/),3541.05 (/),3392.43 (/),2978.57 (/),3365.5 (/)
3,HPI-decomp1,609.71 (459.25 - 760.17),524.15 (425.15 - 623.14),563.27 (442.34 - 684.19),699.91 (520.3 - 879.52),651.5 (374.36 - 928.65)
4,RKIsurv2-arima,225.58 (170.93 - 280.23),221.41 (173.64 - 269.19),219.26 (169.57 - 268.95),263.87 (192.91 - 334.83),214.16 (153.68 - 274.63)
5,RKIsurv2-etsAAA,257.92 (200.62 - 315.22),268.13 (214.58 - 321.68),251.63 (200.27 - 303.0),292.85 (220.01 - 365.7),238.72 (174.55 - 302.89)
6,RKIsurv2-etsAAN,260.09 (203.5 - 316.67),270.12 (218.31 - 321.94),253.3 (202.55 - 304.05),296.09 (222.82 - 369.37),240.77 (177.53 - 304.01)
7,RKIsurv2-etsANN,210.52 (161.48 - 259.55),199.39 (158.17 - 240.6),204.91 (160.98 - 248.84),250.04 (185.14 - 314.94),202.2 (144.85 - 259.55)


**Table 3: (2023) Comparison of forecasting model (submitted and baseline) performances in terms of WIS for the 4 different forecast horizons and the overall performance. Here, the mean and 95% CI of the computed WIS scores is shown.**

In [6]:
tab3

,model,complete score (CI),week 1 score (CI),week 2 score (CI),week 3 score (CI),week 4 score (CI)
0,RKIsurv2-arima,35.75 (29.17 - 42.33),36.88 (28.78 - 44.98),39.2 (32.57 - 45.84),39.9 (35.4 - 44.4),38.58 (35.35 - 41.81)
1,RKIsurv2-etsAAA,41.42 (32.97 - 49.87),40.4 (30.91 - 49.88),46.13 (36.49 - 55.78),47.59 (41.16 - 54.01),48.29 (44.74 - 51.84)
2,RKIsurv2-etsAAN,46.5 (37.02 - 55.97),46.57 (35.77 - 57.36),50.49 (40.45 - 60.53),52.66 (45.4 - 59.92),54.25 (50.14 - 58.37)
3,RKIsurv2-etsANN,29.93 (24.05 - 35.8),32.05 (23.86 - 40.24),34.48 (27.09 - 41.86),31.96 (27.67 - 36.25),30.45 (26.26 - 34.65)


In [3]:
%matplotlib agg
create_figures_baseline_comparison(results, baseline="RKIsurv2-arima")

## Performance over time
The forecasting performance of each individual model in terms of relative WIS for each time point a forecast was submitted is shown. The performance of the submitted model (blue lines) is compared to a baseline model (green lines). The average WIS per forecast date over all counties is shown for the four different forecasting horizons. An ARIMA model was chosen as the baseline.

In [19]:
figures = glob.glob("./docs/figures/baseline_*.png")
models = [fig.split("baseline_")[1][:-4] for fig in figures]
models_dict = {models[i]: f'<img src="{fig}" alt="weighted interval score comparison {models[i]}" />' for i, fig in enumerate(figures)} 

for dm in ['HHI-seq2seq4096d14','HHI-seq2seq4096d21', 'HHI-seq2seq4096d28']:
    del models_dict[dm]
    
display(HTML(generate_tabbed_content(models_dict)))

## Weighted interval score composition

In [ ]:
%matplotlib agg  

figure_wis_composition(results, "Composition of WIS (Year 2021)", max_date="2021-09-01",)

save_current("docs/figures/wis_composition_21.png")

In [3]:
%matplotlib agg  

figure_wis_composition(results, "Composition of WIS (Year 2022)", min_date="2022-01-01", max_date="2022-12-31",
                       drop_models=["HHI-seq2seq4096d14", "HHI-seq2seq4096d21", "HHI-seq2seq4096d28"])

save_current("docs/figures/wis_composition_22.png")

In [4]:
figure_wis_composition(results, "Composition of WIS (Year 2023)", min_date="2023-01-01",
                       drop_models=["HHI-seq2seq4096d14", "HHI-seq2seq4096d21", "HHI-seq2seq4096d28"])

save_current("docs/figures/wis_composition_23.png")

## Performance contribution
The three components of the WIS are for each individual model and each time point for which a forecast was submitted is shown. The components of the WIS are penalties for over-prediction (red), under-prediction (green) and for lack of sharpness of the predictive distribution (blue). 

In [7]:
year_dict = {"2023": '<img src="figures/wis_composition_23.png" alt="Weighted interval score composition 2023"/>',
             "2022": '<img src="figures/wis_composition_22.png" alt="Weighted interval score composition 2022"/>',
             "2021": '<img src="figures/wis_composition_21.png" alt="Weighted interval score composition 2021"/>'}


display(HTML(generate_tabbed_content(year_dict)))

# Relative scores
The Relative WIS (column rel_wis) is a relative measure of forecast performance which takes into account that different teams may not cover the exact same set of forecast targets (i.e., weeks and locations). Loosely speaking, a relative WIS of X means that averaged over the targets a given team addressed, its WIS was X times higher/lower than the performance of the baseline model described in [Cramer et al. (2021)](https://www.medrxiv.org/content/10.1101/2021.02.03.21250974v1). Smaller values are thus better and a value below one means that the model performers above average. The relative WIS is computed using a ‘pairwise comparison tournament’ where for each pair of models a mean score ratio is computed based on the set of shared targets. The relative WIS is the geometric mean of these ratios. Details on the computation can be found in [Cramer et al. (2021)](https://www.medrxiv.org/content/10.1101/2021.02.03.21250974v1).

**Table 4: Relative WIS and MAE scores per model, with ARIMA as the baseline.**

In [7]:
create_table3(results)

,model,n,rel_wis,rel_mae,50% coverage,95% coverage
0,RKIsurv2-arima,31,1.000000,1.000000,0.452169,0.756077
1,HHI-seq2seq4096d14,1,8.185643,7.426270,0.000000,0.000000
2,HHI-seq2seq4096d21,1,8.242499,7.376012,0.000000,0.000000
3,HHI-seq2seq4096d28,1,8.239705,7.365612,0.000000,0.000000
4,HPI-baseMpnnLstm,15,0.512952,0.558373,0.285309,0.635565
5,HPI-decomp1,21,1.905669,1.223774,0.136381,0.304337
6,HHI-ConnstantSpreadTest,1,0.718878,0.181011,0.079663,0.080021
7,RKIsurv2-etsAAA,31,1.082272,1.067369,0.457513,0.749516
8,RKIsurv2-etsAAN,31,1.122335,1.130036,0.467174,0.752777
9,RKIsurv2-etsANN,31,0.952145,0.863889,0.410985,0.687796


## Performance in regions
The forecasting performance of each individual model in terms of average relative WIS for each state of Germany.

In [ ]:
plt.ioff()
regional_heatmap(results)
save_current("docs/figures/regional_heatmap.png")

![Heatmap of relative WIS over regions and models](./figures/regional_heatmap.png)

## Coverage probability
Shown here is the coverage of three different confidence intervals (50%, 80%, 95%) over time for each model. This shows how well calibrated the models are. Models are perfectly calibrated when the forecasted  x% confidence interval covers the true value x% of the time, e.g. the true value will be in the 50% confidence interval for half of the forecasts. A too low coverage means that confidence intervals are too narrow on average. A too high coverage means that confidence Intervals are too wide on average.

In [6]:
%matplotlib agg  

#TODO possibly provide model independent x axis
results["month"] = np.datetime_as_string(results["target"], "M")
grouped = results.groupby(["month", "model"])
coverage_cols = [col for col in list(results.columns) if "within_" in col]
coverage = (grouped.sum()/grouped.count())[coverage_cols].reset_index()
plot_coverage_probability(coverage, temporal_col="month")

![Coverage probabilities per model](./figures/coverage.png)

**Fig. 5: Coverage over time for each model and three different prediction intervals (50%, 80%, 95%). The dashed line shows the expected coverage for the given prediction interval.**